In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dileep070/heart-disease-prediction-using-logistic-regression")

print("Path to dataset files:", path)

In [ ]:
import os
os.listdir(path)

In [3]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [ ]:
df = pd.read_csv(f'{path}/framingham.csv')
df.head()

In [ ]:
df.shape

In [8]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df['male'].unique()

In [ ]:
df['TenYearCHD'].unique()

In [13]:
y = df['TenYearCHD'].values
X = df.drop('TenYearCHD', axis=1)

In [ ]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

In [ ]:
# build mask
gender_0_mask = df['male'] == 0
gender_1_mask = df['male'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

In [16]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

In [18]:
results_df = pd.concat(results_list, ignore_index=True)

In [19]:
result_path = './results/K8_result.xlsx'
results_df.to_excel(result_path, index=False)

In [ ]:
df = pd.read_excel(result_path)
df.head()

In [ ]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)
